# Welcome to the overview of medusa.bci.erp_spellers module!

This module contains high level classes and functions specifically designed
for ERP-based spellers. In this notebook, you will learn about the main
features, function and classes of the module through illustrative examples
which will show you the power of the included tools.

In this notebook you will learn:
    - What is an ERP-based speller
    - Download an open ERP-speller dataset and explore the files
    - Create an instance of ERPSpellerDataset
    - Use important functions of the module
    - Train and test an ERPModel class based on EEG-Inception
    - Train and test a ControlStateModel class based on EEG-Inception

Do not forget to check the documentation if you do not understand something!

## Introduction

ERP-based spellers are


## Imports

Import the modules that will be used in this notebook

In [1]:
from medusa import meeg_standards
from medusa import data_structures
from medusa.bci import erp_spellers
import glob, os
from tabulate import tabulate
import numpy as np

## Download the dataset

As strong supporters of open science, we have released and adapted some
valuable datasets that can be very useful for researchers and practitioners.
These datasets can be downloaded manually from www.medusa.com/datasets/ or
using a simple API. In this case, we will use the API. Run the following cell
to download the GIB-UVa ERP dataset [1].

Each file is an instance of medusa.data_structures.Recording. This class
contains the information of the performed experiment and the registered
biosignals. In this case, we will assume that the recording contains an
instance of medusa.data_structures.ERPSpellerData, which is the default class
for this experiment. Additionally, the recording must contain a
medusa.data_structures.EEG instance.

In [ ]:
# TODO: Download dataset
# dataset_folder = os.getcwd()

## ERPSpellerDataset class

This class is used to define an ERPSpeller dataset, which contains all the
necessary information to work with the functions and classes of the module and
checks for common errors.

First, we have to define the channel set of the dataset. The signals added
to the dataset will be adapted to this channel set, discarding the rest of EEG
channels. In addition, the channels will be reordered if necessary. This
avoids errors in heterogeneous datasets, and, believe me, saves tons of time
wasted debugging machine learning algorithms. In this case, we will use 4 EEG
channels: Fz, Cz, Pz, and Oz.

We also have to define other parameters. The sample rate of the recordings
is set to 256 Hz (if a file has different sample rate, it will throw  an error).
We also define the keys to find the target biosignal (EEG) and experiment data
(ERPSpellerData) attributes in the recording class, a well as the experiment
mode, which is  set to train because we will use this dataset to train a
model later. Check the documentation to understand the details of this and
more parameters.

In [2]:
cha_set = meeg_standards.EEGChannelSet()
cha_set.set_standard_channels(l_cha=['Fz', 'Cz', 'Pz', 'Oz'])
dataset = erp_spellers.ERPSpellerDataset(channel_set=cha_set,
                                         fs=256,
                                         biosignal_att_key='eeg',
                                         experiment_att_key='erpspellerdata',
                                         experiment_mode='train')

## Add recordings to the dataset

Now, we have to add the recordings to the dataset. With this purpose, we read
the files that were downloaded and use the function add_recordingsof our
dataset. Note that this function admits instances of
medusa.data_structures.Recording or a list of paths. For convinience, we will
 use the second option in this case.

In [3]:
folder = 'E:/Eduardo/PythonProjects/medusa/medusa-platform/data'
file_pattern = '*.rcp.bson'
files = glob.glob('%s/%s' % (folder, file_pattern))
dataset.add_recordings(files)

## Explore some functions and classes

Once we have defined our dataset, we can start to play! First, we extract the
ERP features using the function extract_erp_features_from_dataset. This function
returns the ERP epochs, and a variable which keeps track of all the useful
information that allow command decoding and control state detection. Check
the documentation of ERPSpellerData to know the meaning of each of these
variables. Afterwards, we simulate the scores of a classifier using the
labels to check  the decoding  functions. Thus, we expect 100% accuracy for
command decoding and control  state detection tasks. However, we introduced
some errors to check that  everything works.

In [30]:
# Extract ERP features
features, track_info = erp_spellers.extract_erp_features_from_dataset(dataset)

data_exploration = [
    ['Runs', np.unique(track_info['run_idx']).shape[0]],
    ['Epochs', features.shape[0]],
    ['Target', np.sum(track_info['erp_labels']==1)],
    ['Non-target', np.sum(track_info['erp_labels']==0)]
]
print('\nData exploration: \n')
print(tabulate(data_exploration))

# Check command decoding
selected_commands, selected_commands_per_seq, cmd_scores = \
    erp_spellers.decode_commands(track_info['erp_labels'],
                                 track_info['paradigm_conf'],
                                 track_info['run_idx'],
                                 track_info['trial_idx'],
                                 track_info['matrix_idx'],
                                 track_info['level_idx'],
                                 track_info['unit_idx'],
                                 track_info['sequence_idx'],
                                 track_info['group_idx'],
                                 track_info['batch_idx'])

# Introduce error in trial 0 and check accuracy
selected_commands[0][0][0][1] = 2
cmd_acc = erp_spellers.command_decoding_accuracy(
    selected_commands,
    track_info['spell_target']
)
print('\nCommand decoding accuracy:\n')
print('All sequences: %.2f %%' % (cmd_acc * 100))

# Introduce error in trial 0 sequence 14 and check accuracy
selected_commands_per_seq[0][0][0][14][1] = 2
cmd_acc_per_seq = erp_spellers.command_decoding_accuracy_per_seq(
    selected_commands_per_seq,
    track_info['spell_target']
)

table_cmd_acc_per_seq = ['Command decoding accuracy']
table_cmd_acc_per_seq += cmd_acc_per_seq.tolist()
headers = [''] + np.arange(1, 16).tolist()
print(tabulate(table_cmd_acc_per_seq, headers=headers))


Data exploration: 

----------  ----
Runs           4
Epochs      4320
Target       720
Non-target  3600
----------  ----

Command decoding accuracy:

All sequences: 95.83 %


TypeError: 'float' object is not iterable

## ERP speller processing pipeline

Although the previous functions are really powerful, and you can build your
own EEG processing framework for ERP-based spellers easily with them, there is
1 class that combines them all to provide an easy-to-use model, ready to be
used in your projects.

The class ERPModel implements the functions used to control an ERP-based
speller. In other words, it provides the synchronous control of the speller,
implementing all the stages: (1) preprocessing, (2) feature extraction,
(3) feature selection, (4) feature classification, and (5) command decoding.
Currently, it supports 2 classifiers: regularized linear discriminant analysis
(rLDA), and EEG-Inception, each of them requiring different parameters in the
 preprocessing, feature extraction and feature selection stages.

In this tutorial, we will implement a model based on EEG-Inception. By
default, ERPModelSettings are prepared to use this classifier, since it is more
powerful than rLDA [1]. Nevertheless, rLDA may be adequate in some cases, as it
trains faster (especially when no graphic card is available) and requires
less computational resources. Check the documentation to learn how to use
ERPModelSettings to change between the supported models.

In [ ]:
# Instantiate ERPModelSettings and print the summary
settings = erp_spellers.ERPModelSettings()
settings.summary()

# Instantiate ERPSpellerModel
model = erp_spellers.ERPSpellerModel()
model.configure(settings)
# Train model
spell_target, spell_result_per_seq, spell_acc_per_seq = \
    model.fit_dataset(dataset)

print('Train command decoding accuracy per sequence: %s' %
      str(spell_acc_per_seq))

## CSDModel class

There is one limitation that ERPModel cannot solve. ERP-based spellers are,
inherently, synchronous systems. They always make a selection even when the
user is not attending to the stimuli. Nevertheless, this behaviour is not
suitable for real applications, where an asynchronous operation is required.
Do you imagine a web-browser based on an ERP-based speller in which you
cannot read the web page you searched because the system keeps presenting
stimuli?. Despite the futility of a synchronous system, most approaches to
date do not address this issue. In last years, we have worked in it,
achieving quite a success. To know more about this problem and our work in
the field, check references [2], [3] and [4].

The class ERPModel implements the functions used to control an ERP-based
speller. In other words, it provides the synchronous control of the speller,
implementing all the stages: (1) preprocessing, (2) feature extraction,
(3) feature selection, (4) feature classification, and (5) command decoding.

In [ ]:
# Instantiate ERPModelSettings and print the summary
settings = erp_spellers.ERPModelSettings()
settings.summary()

# Instantiate ERPSpellerModel
model = erp_spellers.ERPSpellerModel()
model.configure(settings)
# Train model
cs_target, cs_result_per_seq, cs_acc_per_seq = model.fit_dataset(dataset)

print('Train control state accuracy per sequence: %s' % str(cs_acc_per_seq))

## Conclusion

That's all for now! Now you have a comprehensive picture of the functions and
classes included in the module. As you can see, you can build the full signal
processing pipeline of an ERP-based speller in few code lines using Medusa!

See you in the next tutorial.

## References

Check the following references for extended information about some of the
aspects of this tutorial:

1. Santamaría-Vázquez, E., Martínez-Cagigal, V., Vaquerizo-Villar, F., &
Hornero, R. (2020). EEG-Inception: A Novel Deep Convolutional Neural Network for
Assistive ERP-based Brain-Computer Interfaces. IEEE Transactions on Neural
Systems and Rehabilitation Engineering.
2. Santamaría-Vázquez, E., Martínez-Cagigal, V., Gomez-Pilar, J., & Hornero,
R. (2019). Asynchronous Control of ERP-Based BCI Spellers Using Steady-State
Visual Evoked Potentials Elicited by Peripheral Stimuli. IEEE Transactions on
Neural Systems and Rehabilitation Engineering, 27(9), 1883-1892.
3. Martínez-Cagigal, V., Santamaría-Vázquez, E., & Hornero, R. (2019). 
Asynchronous control of P300-based brain–computer interfaces using sample 
entropy. Entropy, 21(3), 230.

